In [10]:
from google.colab import drive
drive.mount('/content/drive')
% cd drive/MyDrive/thesis
! git clone https://www.github.com/pafitis/thesis

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/thesis
Cloning into 'thesis'...
remote: Enumerating objects: 129, done.
remote: Counting objects: 100% (129/129), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 129 (delta 57), reused 114 (delta 42), pack-reused 0
Receiving objects: 100% (129/129), 15.89 MiB | 14.21 MiB/s, done.
Resolving deltas: 100% (57/57), done.
Checking out files: 100% (86/86), done.


In [15]:
! pip install spacy-nightly

     |████████████████████████████████| 12.7MB 235kB/s 
     |████████████████████████████████| 51kB 7.2MB/s 
     |████████████████████████████████| 9.1MB 21.7MB/s 
     |████████████████████████████████| 624kB 33.6MB/s 
     |████████████████████████████████| 460kB 40.2MB/s 
ERROR: spacy 2.2.4 has requirement catalogue<1.1.0,>=0.0.7, but you'll have catalogue 2.0.4 which is incompatible.
ERROR: spacy 2.2.4 has requirement srsly<1.1.0,>=1.0.2, but you'll have srsly 2.4.1 which is incompatible.
ERROR: spacy 2.2.4 has requirement thinc==7.4.0, but you'll have thinc 8.0.7 which is incompatible.
  Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0


In [2]:
!python -m spacy download en_core_web_trf

2021-07-14 18:16:00.493244: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 459.7MB 1.3MB/s 
     |████████████████████████████████| 12.8MB 4.0MB/s 
     |████████████████████████████████| 2.3MB 36.5MB/s 
     |████████████████████████████████| 1.0MB 37.4MB/s 
     |████████████████████████████████| 3.3MB 30.2MB/s 
     |████████████████████████████████| 901kB 32.2MB/s 
  Created wheel for en-core-web-trf: filename=en_core_web_trf-3.0.0-cp37-none-any.whl size=459708990 sha256=2673a69611e94f5986bfb55317270f7b1e911469f09ca37277985a5d5bbf86d2
  Stored in directory: /tmp/pip-ephem-wheel-cache-9rszr7v4/wheels/a0/7f/ba/6479bde5ccefcfc6b78bd000ea0881a6b218cb8f6b3ea8993c
Successfully built en-core-web-trf
  Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
✔ Download and installation successful
You can now load the package

In [11]:
import sys
sys.path.append('/content/drive/MyDrive/thesis')

In [13]:
import pandas as pd
import numpy as np
import pickle, logging, spacy, sys, os, json, requests
import matplotlib.pyplot as plt

from helpers.classes import Collection
from tqdm import tqdm
from bs4 import BeautifulSoup
from datetime import datetime

from helpers.cloze_generation import \
    generate_clozes_from_point, named_entity_answer_generator as ne_answer_generator, noun_phrase_answer_generator as np_answer_generator

from helpers.language_modelling import \
    find_nth_substring, multitoken_prediction, check_model, run_language_model

ImportError: ignored

In [ ]:
DATASETS = os.listdir('datasets')

with open('pickles/collection_20210624_194932.pkl', 'rb') as f:
    collection = pickle.load(f)

# I should completely remove the other bulletins
# for now this has to do...
bulletin_names = list(collection.bulletins.keys())
# target bulletins are those with usable related datasets and main points
target_bulletins = []
dictionary = dict()
for bulletin in bulletin_names:
    if len(collection.bulletins.get(bulletin).get('main-points')) and len(collection.bulletins.get(bulletin).get('related-datasets')) > 0:
        target_bulletins.append(bulletin)



with open('pickles/dataset_20210625_184837.pkl', 'rb') as f:
    df = pickle.load(f)

In [ ]:
df.head()

,bulletin,type,point,data
0,businessindustryandtrade/business/businessserv...,date_and_percent,"In 2019, approximate gross value added at basi...",[/businessindustryandtrade/business/businessse...
1,businessindustryandtrade/business/businessserv...,date_and_percent,"The non-financial services sector, which accou...",[/businessindustryandtrade/business/businessse...
2,businessindustryandtrade/business/businessserv...,date_and_percent,Total turnover and purchases of the UK non-fin...,[/businessindustryandtrade/business/businessse...
3,businessindustryandtrade/business/businessserv...,date_and_percent,"Out of the 12 UK regions, 8 regions experience...",[/businessindustryandtrade/business/businessse...
4,businessindustryandtrade/business/businessserv...,date_and_percent,"West Midlands, Yorkshire and The Humber, Scotl...",[/businessindustryandtrade/business/businessse...


# T5

In [ ]:
from transformers import T5Config, T5ForConditionalGeneration, T5Tokenizer

device = 'cuda:0' if torch.cuda.is_available else 'cpu'
config = T5Config.from_pretrained('t5-large')
model = T5ForConditionalGeneration.from_pretrained('t5-large').to(device)
tokenizer = T5Tokenizer.from_pretrained('t5-large')

In [ ]:
test_df = df[:3]

In [ ]:
test_df

,bulletin,type,point,data
0,businessindustryandtrade/business/businessserv...,date_and_percent,"In 2019, approximate gross value added at basi...",[/businessindustryandtrade/business/businessse...
1,businessindustryandtrade/business/businessserv...,date_and_percent,"The non-financial services sector, which accou...",[/businessindustryandtrade/business/businessse...
2,businessindustryandtrade/business/businessserv...,date_and_percent,Total turnover and purchases of the UK non-fin...,[/businessindustryandtrade/business/businessse...


In [ ]:
results, entity_set, entities = run_language_model(test_df, model, tokenizer, True)

100%|██████████| 3/3 [00:12<00:00,  4.02s/it]


In [ ]:
results

[[(' 2019',
   0.535437285900116,
   '2019',
   0,
   'ccf75a2c157eacc5253f0e8a55b1ded9f5386d58'),
  (' £88 USD. USD billion',
   0.0005502063581090165,
   '£1,313.9 billion',
   0,
   'fc3346f2545ca5059bdb5150b998300c5001bea5'),
  (' £.bn.n',
   0.004800118851458449,
   '£42.8 billion',
   0,
   '81aa32270d547a368e7b806646c72069ba4fac43'),
  (' (aVA ',
   0.09505190574222439,
   '3.4%',
   0,
   'f308eb661ad1d6c8021cf9e22d8e8c7d9c5ff50e'),
  (' 2018',
   0.8733205199241638,
   '2018',
   0,
   '0a0bcf4b990f1926bc87fa662badadaf7e23d5fa')],
 [],
 [(' £,,,,, billion',
   0.49165046830007253,
   '£4,101.5 billion',
   2,
   '0ff2c435e1d192045540e12b44315fa61592a451'),
  (' £ in in billion in 2019 2020,',
   0.02682128286122674,
   '£2,761.4 billion',
   2,
   '1226559ad975ee3d495c004eb469d294d87bfd5d'),
  ('. sterling pounds billion sterling',
   0.06919503927726445,
   '£70.5 billion',
   2,
   '8444327426fe4840ba8daa1f5058bba3bee0eb42'),
  (' (.% respectively',
   0.15373660986963208,
 

- need to sort out how the answer_given / final_answer is sorted because it seems its wrong
- should think about all these repetitive predictions (respectively x3 etc)
- should fix that trailing whitespace before mask